In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
import time
import pandas as pd
import random
import time
from datetime import datetime, timedelta
import logging
import logging.config
import csv
import os
import urllib.parse

logging.config.fileConfig('logger.config')

logger = logging
warn = logger.warning

In [2]:
def scraping(url):
    global logger
    
    user_agent_list = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
    ]
    # Create a pool of proxies 

    try:
        
        user = random.choice(user_agent_list)
        headers={"accept-language": "en-US,en;q=0.9",
         "accept-encoding": "gzip, deflate, br",
         "User-Agent":user,
         "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7"}
        
        # Send an HTTP request to the URL
        response = requests.get(url, headers=headers)
        # Check if the request was successful
        if response.status_code == 200:
            # Check if the response content type is JSON
            if 'application/json' in response.headers.get('Content-Type', ''):
                # Return the response content as a string
                return response.content.decode('utf-8')
            else:
                # Return None if the response content type is not JSON
                return None
    except requests.RequestException as e:
        print("Error fetching product data:", e)
    return None

In [3]:
with open('../json/seasons.json','r') as file:
    seasons_data = json.load(file)

In [27]:
seasons_json = []

for i in seasons_data:
    base = i['link']
    url = base+"/match-schedule-fixtures-and-results"
    
    html_soup = scraping(url)
    series_json = (html_soup.find('script', {'id': '__NEXT_DATA__'})).contents[0]
    decoded_data = json.loads(series_json)
    seasons_json.append(decoded_data)
    
    time.sleep(10)

In [16]:
decoded_data = json.loads(series_json)

series_data = decoded_data['props']['appPageProps']['data']['series']
matches_data = decoded_data['props']['appPageProps']['data']['content']['matches']

In [17]:
series_id = series_data['id']
season = series_data['season']
year = series_data['year']
location = series_data['description']
start_date = (datetime.fromisoformat(series_data['startDate'])).date().isoformat()
end_date =  (datetime.fromisoformat(series_data['endDate'])).date().isoformat()
total_teams = series_data['totalSquads']

In [18]:
series_id,year,location,start_date,end_date,total_teams

(4162, 2009, 'in England', '2009-06-05', '2009-06-21', 12)

In [19]:
# Season matchs detail
i = matches_data[0]

season_id = i['series']['id']
match_id = i['id']
slug = i['slug']
title = i['title']
time_of_day = i['floodlit']
date = (datetime.fromisoformat(i['startDate'])).date().isoformat()
time = ((datetime.fromisoformat(i['startTime']))+timedelta(hours=5,minutes=30)).time().isoformat()
status = i['status']
status_desc = i['statusText']
winner_team_id = i['winnerTeamId']
toss_winner_team_id = i['tossWinnerTeamId']
toss_choice = i['tossWinnerChoice']
ground_id = i['ground']['id']
team_1_id = i['teams'][0]['team']['id']
team_2_id = i['teams'][1]['team']['id']
team_1_score = i['teams'][0]['score']
team_1_scoreInfo = i['teams'][0]['scoreInfo']
team_2_score = i['teams'][1]['score']
team_2_scoreInfo = i['teams'][1]['scoreInfo']

In [20]:
season_id, match_id, slug, title, time_of_day, date, time, ground_id

(4162,
 18581,
 'england-vs-netherlands-1st-match-group-b',
 '1st Match, Group B',
 'daynight',
 '2009-06-05',
 '22:00:00',
 10)

In [21]:
# ground details

ground = i['ground']
ground_id = ground['id']
ground_name = ground['name']
ground_slug = ground['slug']
ground_img = ground['image']['url']
town_id = ground['town']['id']
ground_capacity = ground['capacity']


In [22]:
# town details

town = ground['town']
town_id = town['id']
town_name = town['name']
country_id = ground['country']['id']


In [23]:
# Country details

country_id = ground['country']['id']
country_name = ground['country']['name']
country_abbrev = ground['country']['abbreviation']

In [8]:
def json_to_csv(json_file,csv_file):
    
    with open(json_file, 'r') as f:
        json_data = json.load(f)

    # Write the JSON data to a CSV file
    with open(csv_file, 'w', newline='') as f:
        fieldnames = json_data[0].keys()
        writer = csv.DictWriter(f, fieldnames=fieldnames)

        writer.writeheader()
        for row in json_data:
            writer.writerow(row)

In [14]:
decoded_data['props']['appPageProps']

with open('../temp/2009.json','w') as f:
    json.dump(decoded_data['props']['appPageProps']['data'],f)

In [28]:
mat_data = []
for j in seasons_json:
    
    matches_data = j['props']['appPageProps']['data']['content']['matches']

    for i in matches_data:
        season_id = i['series']['id']
        match_id = i['id']
        slug = i['slug']
        title = i['title']
        time_of_day = i['floodlit']
        date = (datetime.fromisoformat(i['startDate'])).date().isoformat()
        time = ((datetime.fromisoformat(i['startTime']))+timedelta(hours=5,minutes=30)).time().isoformat()
        status = i['status']
        status_desc = i['statusText']
        winner_team_id = i['winnerTeamId']
        toss_winner_team_id = i['tossWinnerTeamId']
        toss_choice = i['tossWinnerChoice']
        ground_id = i['ground']['id']
        team_1_id = i['teams'][0]['team']['id']
        team_2_id = i['teams'][1]['team']['id']
        team_1_score = i['teams'][0]['score']
        team_1_scoreInfo = i['teams'][0]['scoreInfo']
        team_2_score = i['teams'][1]['score']
        team_2_scoreInfo = i['teams'][1]['scoreInfo']
        
        dict = {
            'season_id' : season_id,
            'match_id' : match_id,
            'slug' : slug,
            'title' : title,
            'time_of_day': time_of_day,  
            'date' : date,  
            'time' : time,  
            'status' : status,  
            'status_description' : status_desc,  
            'winner_team_id' : winner_team_id,  
            'toss_winner_team_id' : toss_winner_team_id,  
            'toss_choice' : toss_choice,  
            'ground_id' : ground_id,
            'team_1_id' : team_1_id,
            'team_2_id' : team_2_id, 
            'team_1_score' : team_1_score,
            'team_1_score' : team_2_score,
            'team_1_scoreInfo' : team_1_scoreInfo,
            'team_2_scoreInfo' : team_2_scoreInfo
            
             # Additional layer for nested data
        }
        
        mat_data.append(dict)

In [32]:
with open('../json/match.json','w') as f:
    json.dump(mat_data,f)

In [33]:
json_to_csv('../json/match.json','../csv/data.csv')

In [ ]:
matches_data = seasons_json[0]['props']['appPageProps']['data']['content']['matches']

In [41]:
ground_data = []
check_set = set()
for j in seasons_json:
    
    matches_data = j['props']['appPageProps']['data']['content']['matches']

    for i in matches_data:
        ground = i['ground']
        ground_id = ground['id']
        ground_name = ground['name']
        ground_slug = ground['slug']
        # ground_img = ground['image']['url']
        town_id = ground['town']['id']
        ground_capacity = ground['capacity']
        
        g_dict = {
            'Ground ID' : ground_id,  
            'Ground Name': ground_name,  
            'Ground Slug': ground_slug,  
            # 'Image URL'  : ground_img,    
            'Town ID'    : town_id,            
            'Capacity'   : ground_capacity     
        }
        if ground_id not in check_set:
            check_set.add(ground_id)
            ground_data.append(g_dict)
        
with open('../json/ground.json','w') as f:
    json.dump(ground_data,f)

json_to_csv('../json/ground.json','../csv/ground.csv')
    

In [3]:
with open('../temp/all.json','r') as f:
    seasons_json = json.load(f)

In [6]:
town_data = []
check_set = set()

for j in seasons_json:
    
    matches_data = j['props']['appPageProps']['data']['content']['matches']

    for i in matches_data:
        # town details
        
        town = i['ground']['town']
        town_id = town['id']
        town_name = town['name']
        country_id = i['ground']['country']['id']
        
        t_dict = {
            'Town ID' : town_id,  
            'Town Name': town_name,  
            'Country ID': country_id 
        }
        
        # print(town_id)
        if town_id not in check_set:
            check_set.add(town_id)
            town_data.append(t_dict)
        
with open('../json/town.json','w') as f:
    json.dump(town_data,f)

json_to_csv('../json/town.json','../csv/town.csv')

In [64]:
country_data = []
check_set = set()

for j in seasons_json:
    
    matches_data = j['props']['appPageProps']['data']['content']['matches']

    for i in matches_data:
        
        ground = i['ground']
        country_id = ground['country']['id']
        country_name = ground['country']['name']
        country_abbrev = ground['country']['abbreviation']
        
        c_dict = {
            'Country ID' : country_id, 
            'Country Name' : country_name,  
            'Country Abbreviation' : country_abbrev,  
            
        }
        if country_id not in check_set:
            check_set.add(country_id)
            country_data.append(c_dict) 


with open('../json/country.json','w') as f:
    json.dump(country_data,f)

json_to_csv('../json/country.json','../csv/country.csv')

In [6]:
matches_data = seasons_json[0]['props']['appPageProps']['data']['content']['matches']


In [12]:
team = matches_data[0]['teams'][0]['team']
team_id = team['id']
team_name = team['name']
team_abbrev = team['abbreviation']
img_url = team['image']['url']
country_id = team['country']['id']

In [35]:
team_data = []
check_set = set()

for j in seasons_json:
    
    matches_data = j['props']['appPageProps']['data']['content']['matches']
    try:
        for i in matches_data:
            teams = i['teams']
            for team in teams:
                team_id = team['team']['id']
                team_name = team['team']['name']
                team_abbrev = team['team']['abbreviation']
                img_url = team['team']['imageUrl']
                
                
                t_dict = {
                    'Team ID' : team_id, 
                    'Team Name' : team_name,  
                    'Team Abbreviation' : team_abbrev,
                    'Image URL' : img_url,
                    'Country ID': team_id
                    
                }
                if team_id not in check_set and team_id>0:
                    check_set.add(team_id)
                    team_data.append(t_dict) 
    except Exception as e:
        
        print('Error on match data',t_dict,e)
        continue

    with open('../json/team.json','w') as f:
        json.dump(team_data,f)

    json_to_csv('../json/team.json','../csv/team.csv')

In [5]:
matches_data = seasons_json[0]['props']['appPageProps']['data']['content']['matches']
for i in matches_data:
    print(icountry_id = ground['country']['id'])

# Data Store in MySQL

In [2]:
import mysql.connector

# Establish connection to MySQL
try:
    # Replace 'username', 'password', and 'host' with your MySQL credentials
    connection = mysql.connector.connect(
        user='root',
        password='123456789',
        host='localhost',
        database = 't20',
        auth_plugin='mysql_native_password'
    )

    if connection.is_connected():
        print("Connected to MySQL")

        # Cursor to execute SQL queries
        cursor = connection.cursor()


except mysql.connector.Error as e:
    print("Error connecting to MySQL:", e)




Connected to MySQL


In [3]:
cursor.execute(""" show tables """)
cursor.fetchall()


[('countrys',), ('grounds',), ('matches',), ('teams',)]

In [13]:
create_teams_table = """CREATE TABLE IF NOT EXISTS teams (
    id INT PRIMARY KEY,
    name VARCHAR(255),
    abbrv VARCHAR(255),
    img VARCHAR(255)
)"""

cursor.execute(create_teams_table)
connection.commit()

In [ ]:
# Execute the INSERT query with placeholders
cursor.execute("INSERT INTO season VALUES (%s,%s,%s,%s,%s,%s)",(series_id,year,location,start_date,end_date,total_teams))

# Commit the transaction
connection.commit()
print("Data inserted successfully")

In [14]:
check_set = set()

for j in seasons_json:
    
    matches_data = j['props']['appPageProps']['data']['content']['matches']
    try:
        for i in matches_data:
            teams = i['teams']
            for team in teams:
                team_id = team['team']['id']
                team_name = team['team']['name']
                team_abbrev = team['team']['abbreviation']
                if team['team']['imageUrl'] is not None:
                    
                    img_url = "https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_640,q_50/lsci"+team['team']['imageUrl']
                else:
                    img_url = ''

                if team_id not in check_set and team_id>0:
                    check_set.add(team_id)
                    cursor.execute("INSERT INTO teams VALUES (%s,%s,%s,%s)",(team_id,team_name,team_abbrev,img_url)) 
                    # Commit the transaction
                    connection.commit()
        logger.info("Teams data inserted....")
            
    except Exception as e:
        
        print('Error on match data',e)
        continue


In [13]:
create_matches_table = """CREATE TABLE IF NOT EXISTS matches (
      id INT PRIMARY KEY,
      season_id INT,
      slug varchar(255),
      title varchar(255),
      time_of_day varchar(255),
      date date,
      time time,
      status varchar(255),
      status_desc varchar(255),
      winner_team_id INT,
      toss_winner_team_id INT,
      toss_choice INT,
      ground_id INT,
      team_1_id INT,
      team_2_id INT,
      team_1_score varchar(255),
      team_1_score_info varchar(255),
      team_2_score varchar(255),
      team_2_score_info varchar(255)
    )"""

cursor.execute(create_matches_table)
connection.commit()

In [ ]:
check_set = set()
for j in seasons_json:
    
    matches_data = j['props']['appPageProps']['data']['content']['matches']

    for i in matches_data:
        season_id = i['series']['id']
        match_id = i['id']
        slug = i['slug']
        title = i['title']
        time_of_day = i['floodlit']
        date = (datetime.fromisoformat(i['startDate'])).date().isoformat()
        time = ((datetime.fromisoformat(i['startTime']))+timedelta(hours=5,minutes=30)).time().isoformat()
        status = i['status']
        status_desc = i['statusText']
        winner_team_id = i['winnerTeamId']
        toss_winner_team_id = i['tossWinnerTeamId']
        toss_choice = i['tossWinnerChoice']
        ground_id = i['ground']['id']
        team_1_id = i['teams'][0]['team']['id']
        team_2_id = i['teams'][1]['team']['id']
        team_1_score = i['teams'][0]['score']
        team_1_score_info = i['teams'][0]['scoreInfo']
        team_2_score = i['teams'][1]['score']
        team_2_score_info = i['teams'][1]['scoreInfo']
        
        if match_id not in check_set:
            check_set.add(match_id)
            cursor.execute("INSERT INTO matches VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)",
                    (match_id, season_id, slug, title, time_of_day, date, time, status, status_desc, winner_team_id, toss_winner_team_id, toss_choice, ground_id, team_1_id, team_2_id, team_1_score, team_1_score_info, team_2_score, team_2_score_info))

        # Commit the transaction
    connection.commit()
    print("Data inserted successfully")
    

In [15]:
create_town_table = """CREATE TABLE IF NOT EXISTS towns (
      id INT PRIMARY KEY,
      town_name VARCHAR(255),
      country_id INT,
      FOREIGN KEY (country_id) REFERENCES countrys(id)
    )"""

cursor.execute(create_town_table)
connection.commit()


check_set = set()
country_chek = set()
for j in seasons_json:
    
    matches_data = j['props']['appPageProps']['data']['content']['matches']

    for i in matches_data:
        # town details
        
        town = i['ground']['town']
        town_id = town['id']
        town_name = town['name']
        country_id = i['ground']['country']['id']
        
        if town_id not in check_set:
            check_set.add(town_id)
            country_chek.add(country_id)
            cursor.execute("INSERT INTO towns VALUES (%s,%s,%s)",
                    (town_id, town_name, country_id))
            connection.commit()
    logger.info("Town details inserted successfully.....")


In [19]:
create_ground_table = """CREATE TABLE IF NOT EXISTS grounds (
      id INT PRIMARY KEY,
      ground_name VARCHAR(255),
      town_id INT,
      ground_capacity varchar(255) 
      
    )"""

cursor.execute(create_ground_table)
connection.commit()


check_set = set()
for j in seasons_json:
    
    matches_data = j['props']['appPageProps']['data']['content']['matches']

    for i in matches_data:
        ground = i['ground']
        ground_id = ground['id']
        ground_name = ground['name']
        ground_slug = ground['slug']
        # ground_img = ground['image']['url']
        town_id = ground['town']['id']
        ground_capacity = ground['capacity']
        
        if ground_id not in check_set:
            check_set.add(ground_id)
            cursor.execute("INSERT INTO grounds VALUES (%s,%s,%s,%s)",
                    (ground_id, ground_name, town_id, ground_capacity))
            connection.commit()
    logger.info("Ground details inserted successfully.....")

In [12]:
cursor.execute("DROP TABLE t20.teams")
connection.commit()


In [14]:
create_country_table = """CREATE TABLE IF NOT EXISTS countrys (
      id INT PRIMARY KEY,
      name VARCHAR(255),
      abbrev varchar(255)
      
    )
    """

cursor.execute(create_country_table)
connection.commit()


check_set = set()
country_chek = set()
for j in seasons_json:
    
    matches_data = j['props']['appPageProps']['data']['content']['matches']

    for i in matches_data:
        
        ground = i['ground']
        country_id = ground['country']['id']
        country_name = ground['country']['name']
        country_abbrev = ground['country']['abbreviation']
        
        if country_id not in check_set:
            check_set.add(country_id)
            country_chek.add(country_id)
            cursor.execute("INSERT INTO countrys VALUES (%s,%s,%s)",
                    (country_id, country_name, country_abbrev))
            connection.commit()
    logger.info("country details inserted successfully.....")


# Particular Match


In [10]:
per_match_data = []
check_set = set()
for j in seasons_json:
    
    matches_data = j['props']['appPageProps']['data']['content']['matches']

    for i in matches_data:
        season_id = i['series']['objectId']
        match_id = i['objectId']
        for i in range(1,3):
            url = f"https://hs-consumer-api.espncricinfo.com/v1/pages/match/comments?lang=en&seriesId={season_id}&matchId={match_id}&inningNumber={i}&commentType=ALL&sortDirection=ASC&fromInningOver="

            per_match_data.append(url)
            
            print(url)

            

https://hs-consumer-api.espncricinfo.com/v1/pages/match/comments?lang=en&seriesId=335113&matchId=355991&inningNumber=1&commentType=ALL&sortDirection=ASC&fromInningOver=
https://hs-consumer-api.espncricinfo.com/v1/pages/match/comments?lang=en&seriesId=335113&matchId=355991&inningNumber=2&commentType=ALL&sortDirection=ASC&fromInningOver=
https://hs-consumer-api.espncricinfo.com/v1/pages/match/comments?lang=en&seriesId=335113&matchId=355992&inningNumber=1&commentType=ALL&sortDirection=ASC&fromInningOver=
https://hs-consumer-api.espncricinfo.com/v1/pages/match/comments?lang=en&seriesId=335113&matchId=355992&inningNumber=2&commentType=ALL&sortDirection=ASC&fromInningOver=
https://hs-consumer-api.espncricinfo.com/v1/pages/match/comments?lang=en&seriesId=335113&matchId=355993&inningNumber=1&commentType=ALL&sortDirection=ASC&fromInningOver=
https://hs-consumer-api.espncricinfo.com/v1/pages/match/comments?lang=en&seriesId=335113&matchId=355993&inningNumber=2&commentType=ALL&sortDirection=ASC&fr

In [11]:
turl = "https://hs-consumer-api.espncricinfo.com/v1/pages/match/comments?lang=en&seriesId=1298134&matchId=1298179&inningNumber=2&commentType=ALL&sortDirection=ASC&fromInningOver=0"

parsed_url = urllib.parse.urlparse(url)
query_params = urllib.parse.parse_qs(parsed_url.query)

series_id = query_params.get('seriesId')[0]
match_id = query_params.get('matchId')[0]
inning_id = query_params.get('inningNumber')[0]

print("Series ID:", series_id)
print("Match ID:", match_id)
print("Inning ID:", inning_id)



Series ID: 1411166
Match ID: 1415755
Inning ID: 2


In [12]:
with open('../json/match_url.json','w') as f:
    json.dump(per_match_data,f)

In [ ]:
per_match_data

In [9]:
print(os.getcwd())

e:\PROJECT\Database\app


In [10]:
print(os.path.join("a","b"))

a\b


In [4]:
def fileName(url):
    parsed_url = urllib.parse.urlparse(url)
    query_params = urllib.parse.parse_qs(parsed_url.query)

    series_id = query_params.get('seriesId')[0]
    match_id = query_params.get('matchId')[0]
    inning_id = query_params.get('inningNumber')[0]
    
    return str(series_id)+"_"+str(match_id)+"_"+str(inning_id)

In [ ]:
# 127 min - 190

In [5]:
file_path = '../json/match_url.json'  # Update with actual file path
checkpoint_file_path = 'checkpoint.json'  # Update with actual checkpoint file path
output_file = '../alljson'


try:
    with open(checkpoint_file_path, 'r') as f:
        checkpoint_status = json.load(f)
except FileNotFoundError:
    checkpoint_status = {}

# Read JSON data from file


# Example of how to use the checkpoint status
last_processed_index = checkpoint_status.get('last_processed_index', 0)
next_over_process = checkpoint_status.get('next_over_process', 0)

logger.info(f"{last_processed_index},{ next_over_process}")
with open(file_path, 'r') as f:
    data = json.load(f)
    
logger.debug(len(data))


    
# Process JSON data starting from last_processed_index
for i in range(last_processed_index, len(data)):
    
    batch_urls = data[i:i+1]
    for url in batch_urls:
        print("Data Scraping started ......")
        # logger.warning(url)
        
        
        print(type(next_over_process), next_over_process)
        if next_over_process is None:
            next_over_process = 0
        print("TOp check ",type(next_over_process), next_over_process)
            
        while next_over_process is not None:
            
            filename = fileName(url)+"_"+str(next_over_process)+".json"
            # print(url, filename)
            print(filename)
            extract_json = scraping(url + str(next_over_process))
            logger.info(f"{filename}, {last_processed_index}, {next_over_process}")
            if extract_json:
                
                decode_json = json.loads(extract_json)
                
                frominningover = decode_json['nextInningOver']
                
                checkpoint_status['next_over_process'] = frominningover
                
                next_over_process = frominningover
                
                with open(os.path.join(output_file,filename), 'w') as f:
                    json.dump(decode_json['comments'], f)

            time.sleep(3)  # import time  # import time
                    
            with open(checkpoint_file_path, 'w') as f:
                json.dump(checkpoint_status, f)
            
        last_processed_index = i+1
           
    checkpoint_status['last_processed_index'] = i+1
        # Update the checkpoint status
    with open(checkpoint_file_path, 'w') as f:
        json.dump(checkpoint_status, f)
    
    # import time
    # Your processing logic here



Data Scraping started ......
<class 'NoneType'> None
TOp check  <class 'int'> 0
1411166_1415753_1_0.json
1411166_1415753_1_2.json
1411166_1415753_1_4.json
1411166_1415753_1_6.json
1411166_1415753_1_8.json
1411166_1415753_1_10.json
1411166_1415753_1_12.json
1411166_1415753_1_13.json
Data Scraping started ......
<class 'NoneType'> None
TOp check  <class 'int'> 0
1411166_1415753_2_0.json
1411166_1415753_2_2.json
1411166_1415753_2_4.json
1411166_1415753_2_6.json
1411166_1415753_2_8.json
1411166_1415753_2_10.json
Data Scraping started ......
<class 'NoneType'> None
TOp check  <class 'int'> 0
1411166_1415754_1_0.json
1411166_1415754_1_2.json
1411166_1415754_1_4.json
1411166_1415754_1_6.json
1411166_1415754_1_8.json
1411166_1415754_1_10.json
1411166_1415754_1_12.json
1411166_1415754_1_14.json
1411166_1415754_1_16.json
1411166_1415754_1_18.json
1411166_1415754_1_20.json
1411166_1415754_1_21.json
Data Scraping started ......
<class 'NoneType'> None
TOp check  <class 'int'> 0
1411166_1415754_2_0

# Latest match detail extract

In [2]:
def scraping(url):
    global logger
    
    user_agent_list = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
    ]
    # Create a pool of proxies 

    try:
        
        user = random.choice(user_agent_list)
        logger.info("user agent is %s",user)
        headers={"accept-language": "en-US,en;q=0.9",
         "accept-encoding": "gzip, deflate, br",
         "User-Agent":user,
         "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7"}
        
        # Send an HTTP request to the URL
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an exception for HTTP errors

        # Parse the HTML content of the response
        soup = BeautifulSoup(response.text, 'html.parser')

        

        return soup
    except requests.RequestException as e:
        print("Error fetching product data:", e)
        return None

In [3]:
seasons_json = []

base = "https://www.espncricinfo.com//series/icc-men-s-t20-world-cup-2024-1411166"
url = base+"/match-schedule-fixtures-and-results"
print(url)
html_soup = scraping(url)
series_json = (html_soup.find('script', {'id': '__NEXT_DATA__'})).contents[0]
decoded_data = json.loads(series_json)
seasons_json.append(decoded_data)



https://www.espncricinfo.com//series/icc-men-s-t20-world-cup-2024-1411166/match-schedule-fixtures-and-results


In [4]:
decoded_data = json.loads(series_json)

series_data = decoded_data['props']['appPageProps']['data']['series']
matches_data = decoded_data['props']['appPageProps']['data']['content']['matches']

In [5]:
mat_data = []
for j in seasons_json:
    
    matches_data = j['props']['appPageProps']['data']['content']['matches']

    for i in matches_data:
        season_id = i['series']['id']
        match_id = i['id']
        slug = i['slug']
        title = i['title']
        time_of_day = i['floodlit']
        date = (datetime.fromisoformat(i['startDate'])).date().isoformat()
        time = ((datetime.fromisoformat(i['startTime']))+timedelta(hours=5,minutes=30)).time().isoformat()
        status = i['status']
        status_desc = i['statusText']
        winner_team_id = i['winnerTeamId']
        toss_winner_team_id = i['tossWinnerTeamId']
        toss_choice = i['tossWinnerChoice']
        ground_id = i['ground']['id']
        team_1_id = i['teams'][0]['team']['id']
        team_2_id = i['teams'][1]['team']['id']
        team_1_score = i['teams'][0]['score']
        team_1_scoreInfo = i['teams'][0]['scoreInfo']
        team_2_score = i['teams'][1]['score']
        team_2_scoreInfo = i['teams'][1]['scoreInfo']
        
        dict = {
            'season_id' : season_id,
            'match_id' : match_id,
            'slug' : slug,
            'title' : title,
            'time_of_day': time_of_day,  
            'date' : date,  
            'time' : time,  
            'status' : status,  
            'status_description' : status_desc,  
            'winner_team_id' : winner_team_id,  
            'toss_winner_team_id' : toss_winner_team_id,  
            'toss_choice' : toss_choice,  
            'ground_id' : ground_id,
            'team_1_id' : team_1_id,
            'team_2_id' : team_2_id, 
            'team_1_score' : team_1_score,
            'team_1_score' : team_2_score,
            'team_1_scoreInfo' : team_1_scoreInfo,
            'team_2_scoreInfo' : team_2_scoreInfo
            
             # Additional layer for nested data
        }
        
        mat_data.append(dict)

In [6]:
with open("../temp/temp.json","w") as f:
    json.dump(mat_data,f)

In [9]:
json_to_csv("../json/match.json","../csv/matches.csv")
